これを実行するには、**無料**のTesla T4 Google Colab環境で「*Runtime*」を押し、「*Run all*」をクリックしてください！  
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a>  
困ったときはDiscordで質問を！＋ ⭐ <i><a href="https://github.com/unslothai/unsloth">Github</a>でスターもお願いします</i> ⭐
</div>

Unslothを自分のPCにインストールするには、[こちら](https://docs.unsloth.ai/get-started/installing-+-updating)のGithubページにあるインストール手順に従ってください。

この実験では、[データ準備](#Data)、[学習](#Train)、[モデルの実行](#Inference)、[保存方法](#Save)を学びます。


### News

Unslothは現在、Text-to-Speech（TTS）モデルに対応しています。詳しい解説は[こちらのガイド](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning)をどうぞ。

**[Gemma 3N Guide](https://docs.unsloth.ai/basics/gemma-3n-how-to-run-and-fine-tune)**もぜひ読んでみてください。また、新しい**[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)**クオンタイズ手法は、他の量子化法を超える高性能を発揮します！

すべての[モデルアップロード](https://docs.unsloth.ai/get-started/all-our-models)や[ノートブック](https://docs.unsloth.ai/get-started/unsloth-notebooks)はドキュメントでチェックしてください。


### Installation

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    #  [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm==0.8.5.post1

In [ ]:
# @title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm==0.8.5.post1
    #  [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    #  Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" huggingface_hub hf_transfer

    #  vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

### Unsloth

Goal: OpenR1のMathデータセットを使って、`DeepSeek-R1-0528-Qwen3-8B`をGRPO経由で推論モデルに変えること。

さらに、言語検出には`langid`を使います。主な目的はモデルにインドネシア語で推論の過程を出力させることで、その確認に`langid`を使った報酬関数を作成しています。

In [ ]:
!pip install langid -qq

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024 # 長い推論過程にも対応可能です
lora_rank = 32 # ランクが大きいほど賢いが、遅くなる

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-0528-Qwen3-8B",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # LoRA 16bitではFalse
    fast_inference = True, # vLLM高速推論を有効にする
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.7, # メモリ不足の場合にReduceする
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # 0より大きい任意の数を選んでください！推奨値：8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank*2, # *2 は学習を2倍高速化します
    use_gradient_checkpointing = "unsloth", # メモリ使用量を削減します
    random_state = 3407,
)

### GRPO Chat Template

Distill Qwen3 from Deepseekには、モデルの入出力をフォーマットするためのチャットテンプレートがあります。これは、モデルの出力をチャット形式にするために使われ、理由づけのステップも含まれています。モデルはこのテンプレートを使って訓練されているため、必ずこのチャットテンプレートを使わなければなりません。

例を使って、チャットテンプレートがどのように動くか見てみましょう：

In [ ]:
reasoning_start = None
reasoning_end = None
user_token = None
assistant_token = None

for token in tokenizer.get_added_vocab().keys():
    if "think" in token and "/" in token:
        reasoning_end = token
    elif "think" in token:
        reasoning_start = token
    elif "user" in token:
        user_token = token
    elif "assistant" in token:
        assistant_token = token

system_prompt = \
f"""You are given a problem.
Think about the problem and provide your working out.
You must think in Bahasa Indonesia."""
system_prompt

In [ ]:
print(tokenizer.apply_chat_template([
    {"role" : "user", "content" : "What is 1+1?"},
    {"role" : "assistant", "content" : f"<think>I think it's 2.2</think>2"},
    {"role" : "user", "content" : "What is 1+1?"},
    {"role" : "assistant", "content" : f"<think>I think it's 2.2</think>2"},
], tokenize = False, add_generation_prompt = True))

### Data Prep
<a name="Data"></a>

今回はHugging Faceの[Open R1 Math dataset](https://huggingface.co/datasets/open-r1/DAPO-Math-17k-Processed)を使います。もちろん、OpenAIの有名な[GSM8K dataset](https://huggingface.co/datasets/openai/gsm8k)も使えます。

In [ ]:
from datasets import load_dataset
dataset = load_dataset("open-r1/DAPO-Math-17k-Processed", "en", split = "train")
dataset

最初の行を見てみましょう：

In [ ]:
dataset[0]["prompt"]

In [ ]:
dataset[0]["solution"]

GSM8Kでは、ほとんどすべての回答に####が含まれているので、それを抽出します。しかし、Open R1データセットについては、以下の処理をスキップできます。

In [ ]:
def extract_hash_answer(text):
    # if "####" が text に含まれていなければ None を返す
    # return text.split("####")[1].strip()  # テキストを"####"で分割し、2番目の部分の前後の空白を削除して返す
    return text
extract_hash_answer(dataset[0]["solution"])

データセットをマッピングしてみましょう！そして最初の行を見てみます：

In [ ]:
dataset = dataset.map(lambda x: {
    "prompt" : [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": x["prompt"]},
    ],
    "answer": extract_hash_answer(x["solution"]),
})
dataset[0]

推論セクションと回答にマッチする正規表現フォーマットを作成します：

In [ ]:
import re

# オプションでEOSトークンの一致を追加する
solution_end_regex = rf"{reasoning_end}(.*)"

match_format = re.compile(solution_end_regex, re.DOTALL)
match_format

動作を確認しましょう：

In [ ]:
match_format.findall(
    "Let me think!</think>"\
    f"Hence, the solution is 2.",
)

In [ ]:
match_format.findall(
    "<think>Let me think!</think>"\
    f"\n\nHence, the solution is 2",
)

報酬関数をフォーマットにピッタリ合わせて作りたいと思います。成功したら3ポイントの報酬を与えます：

In [ ]:
def match_format_exactly(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # 形式が正確に一致する場合にマッチします！
        if match_format.search(response) is not None: score += 3.0
        scores.append(score)
    return scores

失敗しても、モデルが少しでもフォーマットに従っていれば、各記号を数えて報酬を与えたいです：

In [ ]:
def match_format_approximately(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # キーワードがいくつ見つかったか数えます - 多すぎるとペナルティがあります！
        # 1が出たら、ポイントを加算しよう！

        # <think> に報酬を与える必要はありません、なぜなら常に先頭に付けているからです！
        score += 0.5 if response.count(reasoning_start) == 1 else -1.0
        score += 0.5 if response.count(reasoning_end)   == 1 else -1.0
        scores.append(score)
    return scores

生成された回答を抽出して、報酬を与えたりペナルティを課したりしたいです！また、回答が正解にどれだけ近いかを割合で評価して報酬を与えます：

In [ ]:
def check_answer(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1)
        if (guess := match_format.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    for guess, true_answer in zip(extracted_responses, answer):
        score = 0
        if guess is None:
            scores.append(-2.0)
            continue
        # 正解で5ポイント獲得！
        if guess == true_answer:
            score += 5.0
        # 空白がある場合にマッチしますが、報酬は少なめです
        elif guess.strip() == true_answer.strip():
            score += 3.5
        else:
            # 回答が比率的に近ければ、それも評価します！
            # 例えば、答えがある範囲内にあれば報酬を与える！
            try:
                ratio = float(guess) / float(true_answer)
                if   ratio >= 0.9 and ratio <= 1.1: score += 2.0
                elif ratio >= 0.8 and ratio <= 1.2: score += 1.5
                else: score -= 2.5 # 誤答をペナルティする
            except:
                score -= 4.5 # ペナルティを課す
        scores.append(score)
    return scores

また、答えが単一の数字ではなく、「The solution is $20」のような文章の場合もあります。その場合は20を抽出します。

また、123,456のように数値に含まれるカンマも取り除きます。

In [ ]:
match_numbers = re.compile(
    r".*?[\s]{0,}([-]?[\d\.\,]{1,})",
    flags = re.MULTILINE | re.DOTALL
)
print(match_numbers.findall("  0.34  "))
print(match_numbers.findall("  123,456  "))
print(match_numbers.findall("  -0.234  "))
print(match_numbers.findall("17"))

最後に、思考プロセスをインドネシア語（Bahasa Indonesia）で統一するように試みます。これはDeepSeek R1論文で使われている**language consistency reward**（言語一貫性報酬）の簡単なバージョンです。

In [ ]:
import langid

def get_lang(text: str) -> str:
    if not text:
        return "und"
    lang, _ = langid.classify(text)
    return lang


print(get_lang("Hello, How are you")) # これはenを返すはずです
print(get_lang("Aku berpikir kalau aku adalah kamu")) # これはidを返すはずです
print(get_lang("我在这里")) # これは zh を返すはずです

In [ ]:
import re

def format_and_language_reward_func(completions, **kwargs):
    scores = []

    for completion_item in completions:
        if not completion_item or not isinstance(completion_item[0], dict) or "content" not in completion_item[0]:
            scores.append(-5.0)
            print(f"Warning: Malformed completion item, assigning default low score: {completion_item}")
            continue

        content = completion_item[0]["content"]

        lang = get_lang(content)

        if lang == 'id':
            score = 5.0
        elif lang == 'en':
            score = -3.0
        elif lang == 'zh':
            score = -3.0
        else:
            score = -5.0

        scores.append(score)

    return scores

In [ ]:
prompts = [
    [{"role": "assistant", "content": "What is the result of (1 + 2) * 4?"}],
    [{"role": "assistant", "content": "What is the result of (3 + 1) * 2?"}],
]
completions = [
    [{"role": "assistant", "content": "<think>The sum of 1 and 2 is 3, which we multiply by 4 to get 12.</think><answer>(1 + 2) * 4 = 12</answer>"}],
    [{"role": "assistant", "content": "The sum of 3 and 1 is 4, which we multiply by 2 to get 8. So (3 + 1) * 2 = 8."}],
]
format_and_language_reward_func(prompts=prompts, completions=completions)

ここで、生成された応答と正解を表示するメイン関数を準備します。また、テキストを`float`で数値に変換し、それが同じかどうかを判定する別の報酬関数も用意します。

In [ ]:
global PRINTED_TIMES
PRINTED_TIMES = 0
global PRINT_EVERY_STEPS
PRINT_EVERY_STEPS = 5

def check_numbers(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1)
        if (guess := match_numbers.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    # 数ステップごとにのみ出力する
    global PRINTED_TIMES
    global PRINT_EVERY_STEPS
    if PRINTED_TIMES % PRINT_EVERY_STEPS == 0:
        print(
            '*'*20 + f"Question:\n{question}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}"
        )
    PRINTED_TIMES += 1

    for guess, true_answer in zip(extracted_responses, answer):
        if guess is None:
            scores.append(-2.5)
            continue
        # 数字に変換する
        try:
            true_answer = float(true_answer.strip())
            # 123,456のようなカンマを削除する
            guess       = float(guess.strip().replace(",", ""))
            scores.append(3.5 if guess == true_answer else -1.5)
        except:
            scores.append(0)
            continue
    return scores

プロンプトの長さの上位90％を取り出して、うっかり途中で切れてしまわないようにしましょう！

つまり、長さが上位10％に入るプロンプトは除外します。

In [ ]:
tokenized = dataset.map(
    lambda x: {"tokens" : tokenizer.apply_chat_template(x["prompt"], add_generation_prompt = True, tokenize = True)},
    batched = True,
)
print(tokenizer.decode(tokenized[0]["tokens"]))
tokenized = tokenized.map(lambda x: {"L" : len(x["tokens"])})

import numpy as np
maximum_length = int(np.quantile(tokenized["L"], 0.9))
print("最大長 =", maximum_length)

# 最大長の90%未満のサンプルのみをフィルターする
dataset = dataset.select(np.where(np.array(tokenized["L"]) <= maximum_length)[0])
del tokenized

<a name="Train"></a>
### Train the model

さて、GRPO Trainerとすべての設定を整えましょう！

In [ ]:
max_prompt_length = maximum_length + 1 # 念のため1を追加！
max_completion_length = max_seq_length - max_prompt_length

from vllm import SamplingParams
vllm_sampling_params = SamplingParams(
    min_p = 0.1,
    top_p = 1.0,
    top_k = -1,
    seed = 3407,
    stop = [tokenizer.eos_token],
    include_stop_str_in_output = True,
)

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    vllm_sampling_params = vllm_sampling_params,
    temperature = 1.0,
    learning_rate = 5e-6,
    weight_decay = 0.01,
    warmup_ratio = 0.1,
    lr_scheduler_type = "linear",
    optim = "adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # 学習を滑らかにするために4に増やす
    num_generations = 4, # メモリ不足の場合は減らしてください
    max_prompt_length = max_prompt_length,
    max_completion_length = max_completion_length,
    # num_train_epochs = 1, # 完全な学習実行のために1に設定する
    max_steps = 100,
    save_steps = 100,
    report_to = "none", # Weights & Biasesを使用できます
    output_dir = "outputs",

    # オプションのトレーニングと評価用
    # fp16_full_eval = True,  # fp16による完全評価を有効にする
    # per_device_eval_batch_size = 4,
    # eval_accumulation_steps = 1,  # 評価の累積ステップ数 = 1,
    # eval_strategy = "steps",  # 評価戦略をステップごとに設定する
    # eval_steps = 1,  # 評価ステップ数
)

ではtrainerを実行してみましょう！画面を上にスクロールすると、報酬の表が見えます。目標は`reward`列がだんだん増えていくことです！

行動が起こるまでに150〜200ステップかかることがあります。最初の100ステップは報酬が0になることがほとんどです。気長に待ってくださいね！

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [ ]:
# 任意のトレーニングと評価用
# new_dataset = dataset.train_test_split(test_size = 0.01)  # データセットを1%のテストサイズで分割する

trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        match_format_exactly,
        match_format_approximately,
        check_answer,
        check_numbers,
        format_and_language_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,

    # オプションのトレーニングと評価用
    # train_dataset = new_dataset["train"],  # 訓練データセット
    # eval_dataset = new_dataset["test"],  # 評価用データセットをtestデータに設定する
)
trainer.train()

<a name="Inference"></a>
### Inference
さあ、さっき訓練したモデルを試してみましょう！まずは、GRPOを何も訓練していないモデルから試してみます：

In [ ]:
text = "What is the sqrt of 101?"

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

そして今、GRPOを使ってトレーニングしたばかりのLoRAですが、まずはLoRAを保存しましょう！

In [ ]:
model.save_lora("grpo_lora")

LoRAがちゃんと訓練されているか確認しましょう！

In [ ]:
from safetensors import safe_open

tensors = {}
with safe_open("grpo_lora/adapter_model.safetensors", framework = "pt") as f:
    # AとBの両方がゼロでないことを確認する
    for key in f.keys():
        tensor = f.get_tensor(key)
        n_zeros = (tensor == 0).sum() / tensor.numel()
        assert(n_zeros.item() != tensor.numel())

さて、LoRAを読み込んでテストします。カスタムシステムプロンプトを使わずにテストを行いましたが、これはモデル本来の推論能力にほとんど影響を与えないはずです。

In [ ]:
messages = [
    {"role": "user",   "content": "Solve (x + 2)^2 = 0"},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # 生成のために必須です
    tokenize = False,
)
from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 2048,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_lora"),
)[0].outputs[0].text

output

次に、新しい言語を使うはずのシステムプロンプトでテストしてみましょう：

In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": "Solve (x + 2)^2 = 0"},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # 生成のために追加が必要です
    tokenize = False,
)
from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 2048,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_lora"),
)[0].outputs[0].text

output

システムプロンプトあり、しかしLoRAなしの結果と比較してみましょう

In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": "Solve (x + 2)^2 = 0"},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # 生成に必須です
    tokenize = False,
)
from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 2048,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

20個のサンプルを取って、LoRAを使った場合と使わなかった場合で正しい言語処理の量を比較し、どちらが効果的かを見てみましょう。

In [ ]:
sample_dataset = dataset.shuffle(seed = 3407).select(range(20))
sample_dataset

In [ ]:
with_lora_id_count = 0
without_lora_id_count = 0

print("20サンプルでLoRAありとなしの言語使用を比較：")
print("=" * 60)

for i, sample in enumerate(sample_dataset):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": sample["prompt"][1]["content"]},
    ]

    text = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=False,
    )

    output_with_lora = model.fast_generate(
        text,
        sampling_params=sampling_params,
        lora_request=model.load_lora("grpo_lora"),
    )[0].outputs[0].text

    output_without_lora = model.fast_generate(
        text,
        sampling_params=sampling_params,
        lora_request=None,
    )[0].outputs[0].text

    lang_with_lora = get_lang(output_with_lora)
    lang_without_lora = get_lang(output_without_lora)

    if lang_with_lora == 'id':
        with_lora_id_count += 1
    if lang_without_lora == 'id':
        without_lora_id_count += 1

    # 5サンプルごとに進捗を表示する
    if (i + 1) % 5 == 0:
        print(f"Processed {i + 1}/20 samples...")

print("\n" + "=" * 60)
print("結果:")
print(f"With LoRA - Indonesian responses: {with_lora_id_count}/20 ({with_lora_id_count/20*100:.1f}%)")
print(f"Without LoRA - Indonesian responses: {without_lora_id_count}/20 ({without_lora_id_count/20*100:.1f}%)")
print(f"Improvement: +{with_lora_id_count - without_lora_id_count} Indonesian responses with LoRA")

私たちの推論モデルはずっと優秀です。ただし、1時間程度しかトレーニングしていないので、完璧ではありません。シーケンス長を伸ばしてもっと長くトレーニングすれば、さらに良くなりますよ！

<a name="Save"></a>
### VLLM向けにfloat16で保存する方法

`float16`での直接保存にも対応しています。float16の場合は`merged_16bit`を、int4の場合は`merged_4bit`を選んでください。また、フォールバック用に`lora`アダプターも利用可能です。Hugging Faceのアカウントにアップロードしたい場合は、`push_to_hub_merged`を使いましょう！個人用トークンは https://huggingface.co/settings/tokens から取得できます。

In [ ]:
# 16ビットにマージする
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# 4bitにマージする
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# 単なるLoRAアダプターです
if False:
    model.save_pretrained("model")
    tokenizer.save_pretrained("model")
if False:
    model.push_to_hub("hf/model", token = "")
    tokenizer.push_to_hub("hf/model", token = "")


### GGUF / llama.cpp Conversion
`GGUF` / `llama.cpp`への保存がネイティブ対応になりました！`llama.cpp`をクローンして、デフォルトは`q8_0`で保存します。`q4_k_m`などすべての方式をサポートしています。ローカル保存には`save_pretrained_gguf`、Hugging Faceへアップロードする際は`push_to_hub_gguf`を使ってください。

対応している主な量子化方法（一覧は[Wikiページ](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)で確認できます）：
* `q8_0` - 高速変換。リソースは多めですが概ね問題なし。
* `q4_k_m` - おすすめ。attention.wv と feed_forward.w2 のテンソルの半分は Q6_K、それ以外は Q4_Kを使用。
* `q5_k_m` - おすすめ。attention.wv と feed_forward.w2 のテンソルの半分は Q6_K、それ以外は Q5_Kを使用。

[**NEW**] Ollama向けの微調整と自動エクスポートを試すなら、こちらの[Ollamaノートブック](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)がおすすめです。

In [ ]:
# 8bit Q8_0 に保存する
if False: model.save_pretrained_gguf("model", tokenizer,)
# https://huggingface.co/settings/tokens でトークンを取得するのを忘れないでください！
# hfをあなたのユーザー名に変更してください！
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# 16bit GGUFに保存する
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# q4_k_m GGUF に保存する
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# 複数のGGUFオプションに保存 - 複数必要な場合は格段に速くなります！
if False:
    model.push_to_hub_gguf(
        "hf/model", # hf をあなたのユーザー名に変更してください！
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

今度は、`model-unsloth.gguf` ファイルか `model-unsloth-Q4_K_M.gguf` ファイルを、llama.cpp や Jan、Open WebUI のようなUIベースのシステムで使ってみましょう。Janは[こちら](https://github.com/janhq/jan)から、Open WebUIは[こちら](https://github.com/open-webui/open-webui)からインストールできます。

これで準備完了です！Unslothについて質問があれば、私たちの[Discord](https://discord.gg/unsloth)チャンネルへどうぞ！バグを見つけた場合、最新のLLM情報を追いたいとき、助けが欲しいときやプロジェクトに参加したいときも、気軽にDiscordに参加してくださいね。

その他のリンクはこちら：
1. 自分だけの推論モデルを作ろう – Llama GRPOノートブック [無料Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. ファインチューニング結果をOllamaに保存する方法 [無料ノートブック](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Visionファインチューニング – 放射線画像の活用例 [無料Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. DPO、ORPO、継続的事前学習、会話用ファインチューニングなどのノートブックは、[こちらのドキュメント](https://docs.unsloth.ai/get-started/unsloth-notebooks)をご覧ください！

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  助けが必要ならDiscordへ！ + ⭐️ <i><a href="https://github.com/unslothai/unsloth">GithubでStarもよろしくね</a></i> ⭐️
</div>
